## Energy saved from recycling
<p>Did you know that recycling saves energy by reducing or eliminating the need to make materials from scratch? For example, aluminum can manufacturers can skip the energy-costly process of producing aluminum from ore by cleaning and melting recycled cans. Aluminum is classified as a non-ferrous metal.</p>
<p>Singapore has an ambitious goal of becoming a zero-waste nation. The amount of waste disposed of in Singapore has increased seven-fold over the last 40 years. At this rate, Semakau Landfill, Singapore’s only landfill, will run out of space by 2035. Making matters worse, Singapore has limited land for building new incineration plants or landfills.</p>
<p>The government would like to motivate citizens by sharing the total energy that the combined recycling efforts have saved every year. They have asked you to help them.</p>
<p>You have been provided with three datasets. The data come from different teams, so the names of waste types may differ.</p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/wastestats.csv - Recycling statistics per waste type for the period 2003 to 2017</b>
    </div>
    <div>Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>waste_type: </b>The type of waste recycled.</li>
    <li><b>waste_disposed_of_tonne: </b>The amount of waste that could not be recycled (in metric tonnes).</li>
    <li><b>total_waste_recycle_tonne: </b>The amount of waste that could be recycled (in metric tonnes).</li>
    <li><b>total_waste_generated: </b>The total amount of waste collected before recycling (in metric tonnes).</li>
    <li><b>recycling_rate: </b>The amount of waste recycled per tonne of waste generated.</li>
    <li><b>year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/2018_2019_waste.csv - Recycling statistics per waste type for the period 2018 to 2019</b>
    </div>
    <div> Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>Waste Type: </b>The type of waste recycled.</li>
    <li><b>Total Generated: </b>The total amount of waste collected before recycling (in thousands of metric tonnes).</li> 
    <li><b>Total Recycled: </b>The amount of waste that could be recycled. (in thousands of metric tonnes).</li>
    <li><b>Year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/energy_saved.csv -  Estimations of the amount of energy saved per waste type in kWh</b>
    </div>
<ul>
    <li><b>material: </b>The type of waste recycled.</li>
    <li><b>energy_saved: </b>An estimate of the energy saved (in kiloWatt hour) by recycling a metric tonne of waste.</li> 
    <li><b>crude_oil_saved: </b>An estimate of the number of barrels of oil saved by recycling a metric tonne of waste.</li>
</ul>

</div>
<pre><code>
</code></pre>

In [25]:
#Import Necessary Libraries
!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz, process
import os
from pathlib import Path
import re
import pandas as pd
import numpy as np
from itertools import combinations


Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/fuzzywuzzy'
Consider using the `--user` option or check the permissions.

You are using pip version 18.1, however version 21.1.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 4.2MB/s ta 0:00:011
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /home/repl/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/python_Levenshtein-0.12.2.dist-info'
Consider using the `--user` option or check the permissions.

You are using pip version 18.1, however version 21.1.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [4]:
#Find data
find_data = [*os.scandir()]
find_data


[<DirEntry '.profile'>,
 <DirEntry '.cache'>,
 <DirEntry '.bashrc'>,
 <DirEntry '.bash_logout'>,
 <DirEntry '.ipython'>,
 <DirEntry '.jupyter'>,
 <DirEntry '.ipynb_checkpoints'>,
 <DirEntry '.local'>,
 <DirEntry 'datasets'>,
 <DirEntry '.startup.py'>]

In [5]:
datas = [*os.scandir('datasets')]
print(datas)
print(type(datas))

[<DirEntry 'wastestats.csv'>, <DirEntry 'energy_saved.csv'>, <DirEntry '2018_2019_waste.csv'>]
<class 'list'>


In [6]:
#Import Data
waste_stats_older = pd.read_csv('datasets/wastestats.csv')
waste_stats_newer = pd.read_csv('datasets/2018_2019_waste.csv')
energy_stats = pd.read_csv('datasets/energy_saved.csv')
tuple_df = (waste_stats_older, waste_stats_newer, energy_stats)

In [7]:
for df in tuple_df:
    print(df.head())
    print(df.shape)
    print(df.columns[df.isna().any()].tolist())
    print(df.columns)
    

            waste_type  waste_disposed_of_tonne  total_waste_recycled_tonne  \
0                 Food                   679900                    111100.0   
1      Paper/Cardboard                   576000                    607100.0   
2             Plastics                   762700                     59500.0   
3                  C&D                     9700                   1585700.0   
4  Horticultural waste                   111500                    209000.0   

   total_waste_generated_tonne  recycling_rate  year  
0                       791000            0.14  2016  
1                      1183100            0.51  2016  
2                       822200            0.07  2016  
3                      1595400            0.99  2016  
4                       320500            0.65  2016  
(225, 6)
[]
Index(['waste_type', 'waste_disposed_of_tonne', 'total_waste_recycled_tonne',
       'total_waste_generated_tonne', 'recycling_rate', 'year'],
      dtype='object')
                 W

In [16]:
stats_dict = {'waste_type':'Waste Type',
              'waste_disposed_of_tonne':'Waste Disposed of (tonnes)',
              'recycling_rate': 'Recycling Rate',
             'total_waste_generated_tonne':'Total Generated (\'000 tonnes)',
             'total_waste_recycled_tonne':'Total Recycled (\'000 tonnes)',
             'year':'Year'}
waste_stats_older = waste_stats_older.rename(columns = stats_dict)
print(waste_stats_older['Waste Type'].unique())
print(len(waste_stats_older['Waste Type'].unique()))
print(waste_stats_newer['Waste Type'].unique())
print(len(waste_stats_newer['Waste Type'].unique()))
waste_stats_older.head()

['Food' 'Paper/Cardboard' 'Plastics' 'C&D' 'Horticultural waste' 'Wood'
 'Ferrous metal' 'Non-ferrous metal' 'Used slag' 'Ash & Sludge' 'Glass'
 'Textile/Leather' 'Scrap tyres' 'Others (stones, ceramics & rubber etc.)'
 'Total' 'Others (stones, ceramics & rubber etc)' 'Food waste'
 'Construction Debris' 'Wood/Timber' 'Horticultural Waste' 'Ferrous Metal'
 'Non-ferrous Metals' 'Used Slag' 'Sludge' 'Scrap Tyres' 'Ferrous Metals'
 'Others' 'Construction debris' 'Non-ferrous metals' 'Ash and sludge'
 'Plastic' 'Others (stones, ceramic, rubber, etc.)']
32
['Construction& Demolition' 'Ferrous Metal' 'Paper/Cardboard' 'Plastics'
 'Food' 'Wood' 'Horticultural' 'Ash & Sludge' 'Textile/Leather'
 'Used Slag' 'Non-Ferrous Metal' 'Glass' 'Scrap Tyres'
 'Others (stones, ceramic, rubber, ect)' 'Overall']
15


,Waste Type,Waste Disposed of (tonnes),Total Recycled ('000 tonnes),Total Generated ('000 tonnes),Recycling Rate,Year
0,Food,679900,111100.0,791000,0.14,2016
1,Paper/Cardboard,576000,607100.0,1183100,0.51,2016
2,Plastics,762700,59500.0,822200,0.07,2016
3,C&D,9700,1585700.0,1595400,0.99,2016
4,Horticultural waste,111500,209000.0,320500,0.65,2016


In [21]:
xyz = set(waste_stats_newer['Waste Type'].unique())
abc = set(waste_stats_older['Waste Type'].unique())
print(xyz.difference(abc))
fuzz.WRatio(xyz, abc)

{'Overall', 'Others (stones, ceramic, rubber, ect)', 'Horticultural', 'Non-Ferrous Metal', 'Construction& Demolition'}


NameError: name 'fuzz' is not defined

In [ ]:
for w_type in waste_stats_newer['Waste Type']:
    matches = process.extract(state, waste_stats_older['Waste Type'], limit = waste_stats_older.shape[0] )

In [24]:
for row in df.itertuples(name='Animal'):
    print(row)

Animal(Index='dog', num_legs=4, num_wings=0)
Animal(Index='hawk', num_legs=2, num_wings=2)

TypeError: 'tuple' object is not callable

225

In [ ]:
%%nose

import pandas as pd
import re
import numpy as np

convert_index = lambda x: [re.match('(\d{4})', date).group(0) for date in x.index.values.astype(str)]

test_solution = pd.DataFrame({'year': [2015, 2016, 2017, 2018, 2019],\
                             'total_energy_saved': [3.435929e+09, 2554433400, 2.470596e+09, 2.698130e+09,
       2.765440e+09]}).set_index('year')

def test_project():
    
    # Check whether the answer has been saved and is a DataFrame
    assert 'annual_energy_savings' in globals() and type(annual_energy_savings) == pd.core.frame.DataFrame, \
    "Have you assigned your answer to a DataFrame named `annual_energy_savings`?"
    
    # Check whether they have the right column in their DataFrame
    assert annual_energy_savings.columns.isin(['total_energy_saved']).any(), \
    "Your DataFrame is missing the required column!"
    
    # Check whether they have included the correct index
    assert annual_energy_savings.index.name == 'year', \
    "Your DataFrame is missing the required index!"
    
    # Check whether the values (converted to an integer) contain in the only column are correct
    # and check whether the index is identical
    assert (test_solution.total_energy_saved.astype('int64').values == \
    annual_energy_savings.total_energy_saved.astype('int64').values).all()\
    and convert_index(test_solution) == convert_index(annual_energy_savings), \
    "Your submitted DataFrame does not contain the correct values!"